In [1]:
include("src/model.jl")

Main.CESModel

## Loading in Data

In [2]:
data = CESModel.read_data("I-O_DE2019_formatiert.csv")

Main.CESModel.CESData(84×87 DataFrame
 Row │ Sektoren                           Erz. d. Landwirtschaft, Jagd und Die ⋯
     │ String                             Int64                                ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Erz. d. Landwirtschaft, Jagd und…                                       ⋯
   2 │ Forstwirtschaftl. Erzeugnisse un…
   3 │ Fische, Fischerei- und Aquakultu…
   4 │ Kohle
   5 │ Erdöl und Erdgas                                                        ⋯
   6 │ Erze, Steine und Erden, sonstige…
   7 │ Nahrungs- u. Futtermittel, Geträ…
   8 │ Textilien, Bekleidung, Leder und…
  ⋮  │                 ⋮                                          ⋮            ⋱
  78 │ Bruttolöhne und -gehälter                                               ⋯
  79 │ Sonst.Produktionsabgaben abzgl. …
  80 │ Abschreibungen
  81 │ Nettobetriebsüberschuss
  82 │ Bruttowertschöpfung                                                     ⋯
  

## Setting Elasticities

Es können verschiedene Elastizitäten gesetzt werden, die hier verwendeten sind übernommen aus B&F 2019 
(el. of substitiution of inermediates = 0.0001, el labor/intermediates  = 0.5 el. of consumption = 0.9)

In [3]:
elasticities = CESModel.Elasticities(0.0001, 0.5, 0.9)

Main.CESModel.Elasticities(0.0001, 0.5, 0.9)

## Setting Shocks

Shocks können entweder mit `calculate_investment!` in  1000€ eingegeben werden, oder die einzelnen Elemente im Vektor können auch direkt verändernt werden (z.b `supply_shock[12] = 1.2`), was dann einem Anstieg an demand von 20% entsrpicht.

In [4]:
deman_shock = ones(71)
supply_shock = ones(71)
shocks = CESModel.Shocks(supply_shock,deman_shock) 
#shocks.demand_shock[35] = 1.09
CESModel.calculate_investment!(shocks,data,25000,"Vorb.Baustellen-,Bauinstallations-,Ausbauarbeiten")

1.2048978788971576

## Full labor reallocation

In [5]:
price,quantity = CESModel.solve_ces_model(data,shocks,elasticities)

([0.9414662771283979, 0.6855770600909864, 0.6244054586459459, 0.7988094318208029, 0.7256080001106296, 0.7980692412591355, 0.9413472751142221, 0.8399371285656831, 0.7209376231454443, 0.90652307985496  …  0.9374540563116723, 0.936546135463671, 0.9396953689492518, 0.9447017210433883, 0.9573774851060063, 0.9384808316651674, 0.9357786376989145, 0.9267620543014236, 0.865127100350276, 0.9377494149784265], [0.017822842569161355, 0.002698048243079146, 0.00019236861163893581, 0.0010484615291857283, 0.0005707884229604307, 0.005050754335328762, 0.06459037348601993, 0.00763482332417193, 0.009157820799896372, 0.014907716432191012  …  0.08476074351186007, 0.011993745199906813, 0.0592205471991995, 0.07762445712314178, 0.03495355401150268, 0.011428410192898694, 0.009859295553372092, 0.01451722015964288, 0.0015135389093517664, 0.01793051333348333])

In [6]:
println("Real GDP: $(CESModel.real_gdp(price,quantity,data))")
println("Nominal GDP: $(CESModel.nominal_gdp(price,quantity,data))")


Real GDP: 1.0996241642069242


Nominal GDP: 0.9562693811307954


## No labor reallocation 

Es können verschiedene Funktionen angegeben werden,die zurückgeben wie viel Arbeit in den einzelnen Sektoren verfügbar. Hier in diesem Fall, ist keine zusätzliche Arbeit verfügbar, die funktion könnte aber auch zum Beispiel so aussehen: 
```julia
labor_realloc(data) = min(1.05 * data.labor_share,inv(I - diagm(1 .- data.factor_share) * data.Ω) * (data.consumption_share_gross_output .* ((data.shocks.demand_shock .* data.labor_share) - data.labor_share)) + data.labor_share)
``` 
(maximal 5% der verfügbaren Arbeit kann zusätzlich verwendet werden um der gesteigerten Nachfrage nachzukommen)

In [7]:
labor_realloc(data) = data.labor_share #no labor_reallocation

labor_realloc (generic function with 1 method)

In [8]:
price,quantity = CESModel.solve_ces_model(data,shocks,elasticities,labor_reallocation = labor_realloc)

([0.921147632683455, 0.9270650861003054, 0.9214515885955984, 0.9247787486591621, 0.9207069808903534, 0.927935140771916, 0.9217875982036665, 0.9206552309309203, 0.9440167203450254, 0.9217529350560099  …  0.9254894852218095, 0.924072405321481, 0.9219802359949314, 0.9208790433587618, 0.9225067669391435, 0.9227726066407055, 0.9213228347327255, 0.9240239327822368, 0.9207874619163857, 0.9205055190333035], [0.017682137674283335, 0.0022078507670756217, 0.00016202835691791502, 0.0009671103261321754, 0.00048644246496170023, 0.004662879301863719, 0.06393880795818745, 0.00743290317382111, 0.008321802442640358, 0.01457743944741506  …  0.08421263208622785, 0.011938966500094586, 0.059214649035641796, 0.07763949051050958, 0.03495355401150268, 0.011384446759056449, 0.009805254997263705, 0.014340147029542501, 0.0014341421096760916, 0.017885519107604927])

In [9]:
println("Real GDP: $(CESModel.real_gdp(price,quantity,data))")
println("Nominal GDP: $(CESModel.nominal_gdp(price,quantity,data))")

Real GDP: 1.0804534073601424
Nominal GDP: 0.9280529147110474


Shock every sector and look at difference in impacts (output see the 4 demand_shock csv files: nominal, real, labor reallocation, no labor reallocation):

In [10]:
using LinearAlgebra
using Distributed, SharedArrays
shock_values = LinRange(1.0,1.8,50)
supply_shock = ones(71)

GDP_no_realloc_nominal = SharedArray(zeros(71,50))
GDP_nominal = SharedArray(zeros(71,50))

@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init,labor_reallocation = labor_realloc)
        GDP_no_realloc_nominal[i,k] = CESModel.nominal_gdp(p,q,data)
        #take solutions from before as init values, since solution from next run should be close 
        # to soultion from previous (comlex turns the real values back into complex number to allow for solution)
        init = Complex.([p...,q...])
    end
end

#solve sytem for all 71 sector shocks; solve all at the same time (@distributed)
@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init)
        GDP_nominal[i,k] = CESModel.nominal_gdp(p,q,data)
        init = Complex.([p...,q...])
    end
end

In [ ]:
shock_values = LinRange(1.0,1.8,50)
supply_shock = ones(71)

GDP_no_realloc = SharedArray(zeros(71,50))
GDP = SharedArray(zeros(71,50))

@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init,labor_reallocation = labor_realloc)
        GDP_no_realloc[i,k] = CESModel.real_gdp(p,q,data)
        init = Complex.([p...,q...])
    end
end

@distributed for i in 1:71
    init = Complex.([ones(71)..., data.λ...])
    for (k,shock_value) in enumerate(shock_values)
        demand_shock = ones(71)
        demand_shock[i] = shock_value
        shock = CESModel.Shocks(supply_shock,demand_shock)

        p,q = CESModel.solve_ces_model(data,shock,elasticities,init = init)
        GDP[i,k] = CESModel.real_gdp(p,q,data)
        init = Complex.([p...,q...])
    end
end


<<add mark down please; some test>>

In [ ]:
findmax(mapslices(row -> maximum(collect(LinRange(1,maximum(row),50)) - row),GDP,dims=2))

In [ ]:
using DataFrames, CSV
savetabular("demand_shock_nominal.csv",GDP_nominal)
savetabular("demand_shock_nominal_no_realloc.csv",GDP_no_realloc_nominal)
savetabular("demand_shock.csv",GDP)
savetabular("demand_shock_no_realloc.csv",GDP_no_realloc)

In [ ]:
using Plots
scatter()
for row in eachrow(GDP_no_realloc)
    scatter!(row)
end
scatter!()

In [ ]:
scatter()
for row in eachrow(GDP)
    scatter!(row)
end
scatter!()

In [ ]:
plot()
for row in eachrow(GDP)
    plot!(row -  LinRange(1,maximum(row),50))
end
plot!()

In [ ]:
function savetabular(filname,gdp)
    sectors = data.io.Sektoren[1:71]
    df = DataFrame(gdp[1:end,:]',sectors)
    df[:,:DemandShockAmount] = collect(shock_values)
    CSV.write(joinpath(pwd(),"data",filname),df)
end

In [ ]:
GDP_no_realloc_nominal = GDP_no_realloc_nominal[:,1:50]

In [ ]:
GDP_no_realloc